# Strategy Pattern

## Intent
Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.

## Problem
You have multiple ways to do something:
- Different sorting algorithms (bubble, quick, merge)
- Multiple payment methods (credit card, PayPal, crypto)
- Various compression algorithms (ZIP, RAR, 7z)
- Different routing strategies (fastest, shortest, scenic)

**Without Strategy**: Large conditional statements, hard to extend.

## When to Use
✅ **Use when:**
- Multiple algorithms for same task
- Want to switch algorithms at runtime
- Want to hide algorithm implementation details
- Have lots of conditional statements for different behaviors

❌ **Avoid when:**
- Only one or two algorithms
- Algorithms never change
- Clients shouldn't know about different strategies

## Pattern Structure
```
┌─────────┐        ┌──────────┐
│ Context │───────►│ Strategy │
└─────────┘        └──────────┘
                        ▲
            ┌───────────┼───────────┐
       ┌────┴────┐ ┌────┴────┐ ┌───┴────┐
       │Strategy │ │Strategy │ │Strategy│
       │    A    │ │    B    │ │   C    │
       └─────────┘ └─────────┘ └────────┘
```

## Example 1: Payment Processing (Without Strategy)

**Problem**: Hard to add new payment methods

In [ ]:
# WITHOUT Strategy - Messy conditionals
class PaymentProcessor:
    def process_payment(self, amount, method, details):
        if method == "credit_card":
            print(f"Processing ${amount} via Credit Card")
            print(f"  Card: {details['card_number'][-4:]}")
            # Credit card specific logic
        elif method == "paypal":
            print(f"Processing ${amount} via PayPal")
            print(f"  Email: {details['email']}")
            # PayPal specific logic
        elif method == "bitcoin":
            print(f"Processing ${amount} via Bitcoin")
            print(f"  Wallet: {details['wallet'][:10]}...")
            # Bitcoin specific logic
        # Adding new method requires modifying this class!

# Usage - awkward
processor = PaymentProcessor()
processor.process_payment(100, "credit_card", {"card_number": "1234567890123456"})
processor.process_payment(50, "paypal", {"email": "user@example.com"})

## Implementation: Strategy Pattern

In [ ]:
from abc import ABC, abstractmethod

# Strategy interface
class PaymentStrategy(ABC):
    """Abstract payment strategy."""
    
    @abstractmethod
    def pay(self, amount: float) -> bool:
        """Process payment."""
        pass


# Concrete Strategies
class CreditCardPayment(PaymentStrategy):
    """Credit card payment strategy."""
    
    def __init__(self, card_number: str, cvv: str, expiry: str):
        self.card_number = card_number
        self.cvv = cvv
        self.expiry = expiry
    
    def pay(self, amount: float) -> bool:
        print(f"💳 Processing ${amount:.2f} via Credit Card")
        print(f"   Card ending in {self.card_number[-4:]}")
        # Actual payment processing here
        return True


class PayPalPayment(PaymentStrategy):
    """PayPal payment strategy."""
    
    def __init__(self, email: str):
        self.email = email
    
    def pay(self, amount: float) -> bool:
        print(f"🅿️  Processing ${amount:.2f} via PayPal")
        print(f"   Account: {self.email}")
        # PayPal API call here
        return True


class BitcoinPayment(PaymentStrategy):
    """Bitcoin payment strategy."""
    
    def __init__(self, wallet_address: str):
        self.wallet_address = wallet_address
    
    def pay(self, amount: float) -> bool:
        print(f"₿  Processing ${amount:.2f} via Bitcoin")
        print(f"   Wallet: {self.wallet_address[:10]}...")
        # Blockchain transaction here
        return True


# Context
class ShoppingCart:
    """Shopping cart that uses payment strategy."""
    
    def __init__(self):
        self.items = []
        self.payment_strategy = None
    
    def add_item(self, name: str, price: float):
        self.items.append({"name": name, "price": price})
        print(f"✓ Added {name}: ${price:.2f}")
    
    def get_total(self) -> float:
        return sum(item["price"] for item in self.items)
    
    def set_payment_strategy(self, strategy: PaymentStrategy):
        """Set payment method (can change at runtime!)."""
        self.payment_strategy = strategy
        print(f"\n💰 Payment method set to {strategy.__class__.__name__}")
    
    def checkout(self) -> bool:
        """Process checkout using current strategy."""
        if not self.payment_strategy:
            print("❌ No payment method selected!")
            return False
        
        total = self.get_total()
        print(f"\n📦 Checkout - Total: ${total:.2f}")
        return self.payment_strategy.pay(total)


# Demo
print("=== Shopping Cart Demo ===")
cart = ShoppingCart()
cart.add_item("Laptop", 999.99)
cart.add_item("Mouse", 29.99)
cart.add_item("Keyboard", 79.99)

# Try different payment strategies
print("\n--- Payment Option 1: Credit Card ---")
cart.set_payment_strategy(CreditCardPayment("1234567890123456", "123", "12/25"))
cart.checkout()

print("\n--- Payment Option 2: PayPal ---")
cart.set_payment_strategy(PayPalPayment("user@example.com"))
cart.checkout()

print("\n--- Payment Option 3: Bitcoin ---")
cart.set_payment_strategy(BitcoinPayment("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa"))
cart.checkout()

## Real-World Example: Sorting Strategies

In [ ]:
import time
from typing import List

class SortStrategy(ABC):
    """Abstract sorting strategy."""
    
    @abstractmethod
    def sort(self, data: List[int]) -> List[int]:
        pass


class BubbleSort(SortStrategy):
    """Bubble sort - simple, slow for large datasets."""
    
    def sort(self, data: List[int]) -> List[int]:
        arr = data.copy()
        n = len(arr)
        for i in range(n):
            for j in range(0, n - i - 1):
                if arr[j] > arr[j + 1]:
                    arr[j], arr[j + 1] = arr[j + 1], arr[j]
        return arr


class QuickSort(SortStrategy):
    """Quick sort - efficient for most cases."""
    
    def sort(self, data: List[int]) -> List[int]:
        if len(data) <= 1:
            return data
        
        pivot = data[len(data) // 2]
        left = [x for x in data if x < pivot]
        middle = [x for x in data if x == pivot]
        right = [x for x in data if x > pivot]
        
        return self.sort(left) + middle + self.sort(right)


class MergeSort(SortStrategy):
    """Merge sort - stable, predictable performance."""
    
    def sort(self, data: List[int]) -> List[int]:
        if len(data) <= 1:
            return data
        
        mid = len(data) // 2
        left = self.sort(data[:mid])
        right = self.sort(data[mid:])
        
        return self._merge(left, right)
    
    def _merge(self, left: List[int], right: List[int]) -> List[int]:
        result = []
        i = j = 0
        
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1
        
        result.extend(left[i:])
        result.extend(right[j:])
        return result


class DataProcessor:
    """Processes data using sorting strategy."""
    
    def __init__(self, strategy: SortStrategy = None):
        self.strategy = strategy or QuickSort()
    
    def set_strategy(self, strategy: SortStrategy):
        self.strategy = strategy
    
    def process(self, data: List[int]) -> List[int]:
        start = time.time()
        result = self.strategy.sort(data)
        elapsed = time.time() - start
        
        print(f"{self.strategy.__class__.__name__}: {elapsed*1000:.2f}ms")
        return result


# Demo
import random

data = [random.randint(1, 100) for _ in range(20)]
print(f"Original: {data[:10]}...\n")

processor = DataProcessor()

# Try different strategies
print("Comparing sorting strategies:")
processor.set_strategy(BubbleSort())
result1 = processor.process(data)

processor.set_strategy(QuickSort())
result2 = processor.process(data)

processor.set_strategy(MergeSort())
result3 = processor.process(data)

print(f"\nSorted: {result3[:10]}...")

## Real-World Example: Compression Strategies

In [ ]:
import zlib
import base64

class CompressionStrategy(ABC):
    """Abstract compression strategy."""
    
    @abstractmethod
    def compress(self, data: str) -> bytes:
        pass
    
    @abstractmethod
    def decompress(self, data: bytes) -> str:
        pass


class ZipCompression(CompressionStrategy):
    """ZIP/zlib compression."""
    
    def compress(self, data: str) -> bytes:
        return zlib.compress(data.encode())
    
    def decompress(self, data: bytes) -> str:
        return zlib.decompress(data).decode()


class Base64Compression(CompressionStrategy):
    """Base64 encoding (not real compression, for demo)."""
    
    def compress(self, data: str) -> bytes:
        return base64.b64encode(data.encode())
    
    def decompress(self, data: bytes) -> str:
        return base64.b64decode(data).decode()


class FileArchiver:
    """Archives files using compression strategy."""
    
    def __init__(self, strategy: CompressionStrategy):
        self.strategy = strategy
    
    def archive(self, data: str) -> bytes:
        print(f"Original size: {len(data)} bytes")
        compressed = self.strategy.compress(data)
        print(f"Compressed ({self.strategy.__class__.__name__}): {len(compressed)} bytes")
        print(f"Ratio: {len(compressed)/len(data)*100:.1f}%\n")
        return compressed
    
    def extract(self, data: bytes) -> str:
        return self.strategy.decompress(data)


# Demo
text = "Hello World! " * 50  # Repeated text compresses well

print("=== ZIP Compression ===")
archiver = FileArchiver(ZipCompression())
compressed = archiver.archive(text)
extracted = archiver.extract(compressed)
assert extracted == text

print("=== Base64 Encoding ===")
archiver = FileArchiver(Base64Compression())
compressed = archiver.archive(text)
extracted = archiver.extract(compressed)
assert extracted == text

## Python-Specific: Using Functions as Strategies

Python's first-class functions make strategies simpler!

In [ ]:
# Strategy as functions (Pythonic way)
def calculate_by_quantity(price: float, quantity: int) -> float:
    """Standard pricing."""
    return price * quantity

def calculate_bulk_discount(price: float, quantity: int) -> float:
    """10% discount for 10+ items."""
    total = price * quantity
    if quantity >= 10:
        total *= 0.9
    return total

def calculate_seasonal_discount(price: float, quantity: int) -> float:
    """20% seasonal discount."""
    return price * quantity * 0.8


class Order:
    """Order with pricing strategy."""
    
    def __init__(self, price: float, quantity: int, pricing_strategy):
        self.price = price
        self.quantity = quantity
        self.pricing_strategy = pricing_strategy  # Function!
    
    def calculate_total(self) -> float:
        return self.pricing_strategy(self.price, self.quantity)


# Demo
print("=== Pricing Strategies (Functional) ===")

order1 = Order(10.0, 5, calculate_by_quantity)
print(f"Standard pricing (5 items): ${order1.calculate_total():.2f}")

order2 = Order(10.0, 15, calculate_bulk_discount)
print(f"Bulk discount (15 items): ${order2.calculate_total():.2f}")

order3 = Order(10.0, 5, calculate_seasonal_discount)
print(f"Seasonal discount (5 items): ${order3.calculate_total():.2f}")

# Can even use lambda!
order4 = Order(10.0, 5, lambda p, q: p * q * 0.5)  # 50% off
print(f"Flash sale (5 items): ${order4.calculate_total():.2f}")

## Strategy vs State Pattern

**Strategy**: Client chooses the strategy
```python
cart.set_payment_strategy(PayPalPayment())  # Client decides
```

**State**: Context transitions automatically
```python
order.process()  # State changes from Pending → Paid → Shipped
```

## Advantages & Disadvantages

### ✅ Advantages
1. **Open/Closed Principle**: Add strategies without changing context
2. **Runtime switching**: Change algorithms dynamically
3. **Isolate implementation**: Hide algorithm details
4. **Replace inheritance**: Use composition instead
5. **Easy testing**: Test strategies independently

### ❌ Disadvantages
1. **More classes**: One class per strategy
2. **Client awareness**: Clients must know about strategies
3. **Communication overhead**: Passing data to strategies
4. **Overkill**: For simple cases, might be over-engineering

## When to Use Strategy vs Other Patterns

**Use Strategy when:**
- Multiple algorithms for same task
- Want to swap at runtime
- Algorithms are independent

**Use Template Method when:**
- Algorithm skeleton is fixed
- Only steps vary
- Inheritance-based approach preferred

**Use Command when:**
- Need to queue/log operations
- Need undo/redo
- Operations are requests, not algorithms

## Best Practices

1. **Use functions** in Python when strategies are simple
2. **Default strategy**: Provide sensible default
3. **Strategy factory**: Centralize strategy creation
4. **Immutable strategies**: Make strategies stateless when possible
5. **Document trade-offs**: When to use each strategy

## Related Patterns

- **State**: Similar structure, different intent
- **Template Method**: Alternative for algorithm variation
- **Flyweight**: Share strategy objects
- **Decorator**: Add behavior, Strategy replaces behavior

## Summary

Strategy pattern enables:
- Algorithm family encapsulation
- Runtime algorithm selection
- Independent algorithm variation
- Elimination of conditionals

Perfect for: Payment methods, sorting algorithms, compression, validation, pricing rules.

**Python Tip**: Use functions/lambdas for simple strategies, classes for complex ones!